<a href="https://colab.research.google.com/github/caumannerman/ncm_project/blob/master/%ED%8C%8C%EC%9D%B4%EC%8D%AC_%EA%B3%BC%EC%A0%9C_%EB%A7%88%EB%B2%95%EA%B3%B5%EC%8B%9D_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
pip install pykrx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pykrx import stock
import pandas as pd
import requests
import time
from datetime import datetime

In [ ]:
# EV/EBTIDA 가져오기
# date='211230'
# year='2021/12'
def get_EVEBITDA(code, year):
  # print ("in get_EVEBITDA")
  url = 'https://comp.fnguide.com/SVO2/ASP/SVD_Invest.asp?pGB=1&gicode=A' + code + '&cID=&MenuYn=Y&ReportGB=&NewMenuID=105&stkGb=701'

  fr_page = requests.get(url)
  fr_tables = pd.read_html(fr_page.text)

  temp_df = fr_tables[3]
  temp_df = temp_df.set_index(temp_df.columns[0])
  temp_df = temp_df.loc[['EV/EBITDA계산에 참여한 계정 펼치기']]
  temp_df.index = ['EV/EBITDA']
  temp_df = pd.DataFrame({code:temp_df[year]})
  temp_df = temp_df.transpose()
  # print (temp_df)
  return temp_df

In [ ]:
# ROIC 가져오기
# date='211230'
# year='2021/12'
def get_ROIC(code, year):
  # print ("in get_ROIC")
  url = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A'+ code +'&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'

  fr_page = requests.get(url)
  fr_tables = pd.read_html(fr_page.text)

  temp_df = fr_tables[0]
  temp_df = temp_df.set_index(temp_df.columns[0])
  temp_df = temp_df.loc[['ROIC계산에 참여한 계정 펼치기']]
  temp_df.index = ['ROIC']
  temp_df = pd.DataFrame({code:temp_df[year]})
  temp_df = temp_df.transpose()
  print (temp_df)
  return temp_df

# print (get_ROIC('095570',year))

In [11]:
# 종목코드, 종목명 가져오기
# 2022년 1년간의 종목 수익률을 파악해야 하므로 종목 추출 시점은 2021년 연말이 되어야 함
# 따라서 종목, EV/EBTIDA, ROIC 모두 2021년 4분기 기준값으로 가져오기
date='191230'
year='2019/12'
codes = stock.get_market_ticker_list(date, market='ALL')
corp = []

for code in codes:
  name = stock.get_market_ticker_name(code)
  corp.append([code, name])

df = pd.DataFrame(data=corp, columns=['종목코드', '종목명'])
df_code = df['종목코드']

# EV/EBITDA dataframe 생성 및 적재
# 첫번재 종목이 3월 결산 종목이라 2021년 12월 데이터 값이 홈페이지에 나와있지 않음 -> 삼성전자를 임의로 첫번째 값으로 입력
evebitda_df=get_EVEBITDA('005930',year)

for num, code in enumerate(df_code):  
  # if num==5:
  #     break
  try:
      if num==1:
        return_df=get_EVEBITDA(code,year)
        evebitda_df=pd.concat([evebitda_df, return_df])
        print (evebitda_df)
        evebitda_df=evebitda_df.drop(index='005930')  
      else:
        return_df=get_EVEBITDA(code,year)
        evebitda_df=pd.concat([evebitda_df, return_df])
  except:
      pass
print (evebitda_df)

# ROIC dataframe 생성 및 적재
roic_df=get_ROIC('005930',year)

for num, code in enumerate(df_code):  
  # if num==5:
  #     break
  try:
      if num==1:
        return_df=get_ROIC(code,year)
        roic_df=pd.concat([roic_df, return_df])
        print (roic_df)
        roic_df=roic_df.drop(index='005930')  
      else:
        return_df=get_ROIC(code,year)
        roic_df=pd.concat([roic_df, return_df])
  except:
      pass
print (roic_df)


       EV/EBITDA
005930      6.60
        ROIC
005930  13.2
        ROIC
005930  13.2


In [ ]:
evebitda_df.index

Index(['005930', '068400', '006840', '054620', '090470', '001460', '032040',
       '001040', '079160', '000120',
       ...
       '064090', '079980', '065510', '005010', '084110', '069260', '024060',
       '010240', '003280', '037440'],
      dtype='object', length=1482)

In [ ]:
print(evebitda_df)
print(roic_df)

magic = evebitda_df.join(roic_df,how='left')


#magic = pd.concat([evebitda_df,roic_df],axis=1)
print(magic)

       EV/EBITDA
005930      8.33
068400      4.99
006840     39.71
054620       NaN
090470       NaN
...          ...
069260      8.28
024060   6640.26
010240      3.82
003280     30.83
037440      4.76

[1482 rows x 1 columns]
         ROIC
005930   17.2
068400    3.5
006840   -7.2
054620  -50.2
090470   -8.1
...       ...
069260   26.9
024060   -1.1
010240   23.0
003280   -2.9
037440   11.6

[1478 rows x 1 columns]
       EV/EBITDA   ROIC
000020     16.55   11.3
000040     30.03    1.4
000050      8.33    NaN
000070      6.27    7.9
000080      8.54    7.5
...          ...    ...
900070      3.55    5.7
900080       NaN    NaN
900100     42.44   -1.7
900110     17.11  -15.5
900140    101.71   -2.5

[1484 rows x 2 columns]


In [ ]:
# 여기부터 추가된 코드
# ROIC와 EV/EBITDA 열이 모두 String타입이었는데 이를 float타입으로 바꾼다 ( 대소관계를 따져 정렬해야하므로)
magic22 = magic.astype({'ROIC':'float', 'EV/EBITDA':'float'})
# EV/EBITDA 기준으로 오름차순 정렬
magic22 = magic22.sort_values('EV/EBITDA', ascending=True)
print(magic22)
# EV/EBITDA 기준으로 오름차순 정렬되어있으므로, 해당순서대로 EV/EBITDA rank값을 할당해줌 
magic22['EV/EBITDA_rank'] = [i for i in range(1, len(magic22) + 1)]
print(magic22)

roic_arr = []

# 인덱스( 종목 번호)와 roic값을 tuple형태로 넣어줌 
for idx, row in magic22.iterrows():
    # print("** index name:", idx)
    # print(idx, row["ROIC"])
    roic_arr.append((row["ROIC"], idx))

# roic 값을 기준으로 하여 내림차순 정렬 
roic_arr = sorted(roic_arr, reverse = True)
# 인덱스 (종목번호)가 원래의 순서 그대로 담긴 list를 생성 
iiidx_arr = list(magic22.index)
print(roic_arr)

# 여기부터 ROIC_rank 에 차례대로 값을 담을 것
ROIC_rank_arr = [0] * len(iiidx_arr)
for idx, (now_roic, secu_num) in enumerate(roic_arr):
  # print(iiidx_arr.index(secu_num))
  # print(idx)
  ROIC_rank_arr[iiidx_arr.index(secu_num)] = idx + 1

# 결과적으로 ROIC_rank_arr에 roic 랭크가 담김
print(ROIC_rank_arr)

magic22['ROIC_rank'] = ROIC_rank_arr

print(magic22)

# EBITDA_rank와 ROIC_rank를 합쳐서 result_rank를 새로운 열에 저장 
magic22['result_rank'] = magic22['EV/EBITDA_rank'] + magic22['ROIC_rank']
print(magic22)

magic22.to_excel('Magic(' + '2021_new' + ').xlsx')
# 끗!!!!!!!!!!
# 끗!!!!!!!!!!
# 끗!!!!!!!!!!
# 끗!!!!!!!!!!

        EV/EBITDA   ROIC
024830       1.46   11.9
035890       1.85    NaN
046110       2.02   13.5
004250       2.19    6.9
037400       2.42   12.3
...           ...    ...
140410        NaN -114.2
141070        NaN  -48.0
144620        NaN  -24.7
159910        NaN  -35.0
900080        NaN    NaN

[1484 rows x 2 columns]
        EV/EBITDA   ROIC  EV/EBITDA_rank
024830       1.46   11.9               1
035890       1.85    NaN               2
046110       2.02   13.5               3
004250       2.19    6.9               4
037400       2.42   12.3               5
...           ...    ...             ...
140410        NaN -114.2            1480
141070        NaN  -48.0            1481
144620        NaN  -24.7            1482
159910        NaN  -35.0            1483
900080        NaN    NaN            1484

[1484 rows x 3 columns]
[(nan, '035890'), (597.9, '092130'), (344.8, '001880'), (177.6, '072870'), (90.3, '036190'), (52.0, '048260'), (49.4, '042420'), (44.4, '104460'), (38.8, '013

In [ ]:
# def sortDataFrame(df):
#   code_EE_list = []
#   code_RO_list = []
#    # 일단 df에 EV/EBITDA_rank, ROIC_rank, Final_rank 열을 추가해야한다.
#   df['EV/EBITDA_rank'] = [i for i in range(1, len(df) + 1)]
#   df['ROIC_rank'] = code_RO_list
#   df['Final_rank'] = 0

  
#   # 매개변수로 전달받은 DataFrame의 각 row를 돌며  
#   # ee_list에는 (종목코드(문자열), EV/EBITDA (float)) 와 같이 튜플 형태로 쌓아줌
#   # for idx, row in df.iterrows():
#   #   df['EV/EBITDA_rank']
#   #   code_EE_list.append((row["EV/EBITDA"], idx))
#   #   code_RO_list.append((row["ROIC"], idx))

 
#   # df.loc['095570']['EV/EBITDA_rank'] = (1.1, "lll")
#   return df
#   # return code_EE_list, code_RO_list



In [ ]:
from pykrx import stock

In [ ]:
from pykrx import stock
import pandas as pd
import requests
import time
from datetime import datetime

In [ ]:
from pykrx import stock
import pandas as pd
 
stock_price = stock.get_market_price_change_by_ticker("20220101", "20221230")
print(stock_price)

            종목명      시가      종가    변동폭     등락률        거래량           거래대금
티커                                                                      
095570   AJ네트웍스    5410    5720    310    5.73   35515006   240165143451
006840    AK홀딩스   19300   17200  -2100  -10.88    5895069   109883273500
027410      BGF    5290    4305   -985  -18.62   79220901   339205002890
282330   BGF리테일  145500  210500  65000   44.67   10907706  1930545130101
138930  BNK금융지주    8400    6500  -1900  -22.62  283430714  2064155391561
...         ...     ...     ...    ...     ...        ...            ...
015350     부산가스   84400       0 -84400 -100.00          0              0
009275      신원우   41050       0 -41050 -100.00          0              0
138250    엔에스쇼핑   12700       0 -12700 -100.00          0              0
007630  폴루스바이오팜    1225       0  -1225 -100.00          0              0
153360   하이골드3호    2660       0  -2660 -100.00          0              0

[942 rows x 7 columns]
